In [ ]:
##創資料夾
import os
import datetime
end = datetime.date.today()

def mkdir(path):
    folder = os.path.exists(path)
    #判斷結果
    if not folder:
        #如果不存在，則建立新目錄
        os.makedirs(path)
        print('-----建立成功-----')
    else:
        #如果目錄已存在，則不建立，提示目錄已存在
        print(path+'目錄已存在')
path = str(end) + '_50+6_stock'
mkdir(path)

# 獲取台股資料

In [ ]:
## 資料匯出
import pandas as pd
def OutputCSV(sid, stock_dr):
    Result = path + '/' + sid + '.csv'
    df_SAMPLE = pd.DataFrame.from_dict(stock_dr)
    df_SAMPLE.to_csv(Result, index=True)
    print('成功產出'+ Result)

In [ ]:
import pandas as pd
import datetime
from pandas_datareader import data
import time

# 與yahoo請求，套件路徑因版本不同
#pd.core.common.is_list_like = pd.api.types.is_list_like

start  = datetime.datetime(2000,1,1,0,0,0)
end = datetime.date.today()
def get_taiwan50data():
    path = 'StockList_0728.xlsx' ## 人工建資料
    df = pd.read_excel(path) ## 拿到 台灣50指數成分股
    for sid in df['代號']:
        sid = str(sid)
        TW_stock_dr = data.get_data_yahoo(sid+'.TW', start, end)
        TW_stock_dr = insert_MA(TW_stock_dr)
        TW_stock_dr = insert_KDJ(TW_stock_dr)
        TW_stock_dr = insert_KDX(TW_stock_dr)
        TW_stock_dr = insert_RSI(TW_stock_dr)
        # 合併美股資料
        combine_data(TW_stock_dr, sid)

In [ ]:
def insert_MA(data):
    MA_5days = data['Close'].rolling(5).mean()
    MA_20days = data['Close'].rolling(20).mean()
    MA_240days = data['Close'].rolling(240).mean()
    data['MA_5days'] = MA_5days
    data['MA_20days'] = MA_20days
    data['MA_240days'] = MA_240days
    return data

In [ ]:
def insert_KDJ(data):
    low_list = data['Low'].rolling(window=9).min()
    high_list = data['High'].rolling(window=9).max()
    rsv = (data['Close'] - low_list) / (high_list - low_list) * 100
    data['K_value'] = rsv.ewm(com=2).mean() 
    data['D_value'] = data['K_value'].ewm(com=2).mean()
    data['J_value'] = 3 * data['K_value'] - 2 * data['D_value']
    return data

In [ ]:
def insert_KDX(data):
    data['KDJ_金叉死叉'] = ''
    KDX = data['K_value'] > data['D_value']
    data.loc[KDX[(KDX == True) & (KDX.shift() == False)].index, 'KDJ_金叉死叉'] = '金叉'
    data.loc[KDX[(KDX == False) & (KDX.shift() == True)].index, 'KDJ_金叉死叉'] = '死叉'
    return data

In [ ]:
import numpy as np
import pandas as pd
import talib
def insert_RSI(data):
    data['RSI6'] = talib.RSI(data['Close'],timeperiod=6) 
    data['RSI12'] = talib.RSI(data['Close'],timeperiod=12) 
    return data

# 獲取美股資料

In [ ]:
list_USstock = ['^DJI','^GSPC','^IXIC','^SOX','^NYA']
def get_USAstockdata():
    combine_TW_stock = None
    for sid in list_USstock:
        US_stock_dr = data.get_data_yahoo(sid, start, end)
        US_stock_dr = insert_MA(US_stock_dr)
        US_stock_dr = insert_KDJ(US_stock_dr)
        US_stock_dr = insert_KDX(US_stock_dr)
        US_stock_dr = insert_RSI(US_stock_dr)
        OutputCSV(sid, US_stock_dr)
        print("sleep 10 seconds.")
        time.sleep(10)
        print("printed after 10 seconds.")

# 將美股合併至台股中
def combine_data(TW_stock_dr, TW_sid):
    path = str(end) + '_50+6_stock'
    combine_TW_stock = None
    for sid in list_USstock:
        file_path = path + '/' + sid + '.csv'
        US_stock_dr = pd.read_csv(file_path)
        combine_US_stock = TW_stock_dr.combine_first(US_stock_dr)
        combine_TW_stock = pd.concat([combine_TW_stock,combine_US_stock]).sort_values(by='Date').drop_duplicates()
    OutputCSV(TW_sid, combine_TW_stock)
    print("sleep 10 seconds.")
    time.sleep(10)
    print("printed after 10 seconds.")

In [ ]:
get_USAstockdata()

In [ ]:
get_taiwan50data()